## Librerias

In [44]:
from pyGCS_ import *
import numpy as np
import numpy.ma as ma
import datetime
import struct
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.patches as mpatches
from scipy.stats import kde
from astropy.io import fits #proporciona acceso a los archivos de FITS(Flexible Image Transport System) es un estándar de archivos portátiles 
import sunpy
from sunpy.coordinates.ephemeris import get_horizons_coord
import sunpy.map
from sunpy.map.maputils import all_coordinates_from_map
import json
import random

## Diccionario de lista de combinaciones de parametros

In [67]:
def dic_Parameters(CMElons, CMElats, CMEtilts, heights, ks, angs, nleg, ncirc, ncross):
    list_config = []
    dic_parameters = {}
    combinations = len(CMElons)**6
    # Se crean todas las combinaciones de parametros:
    for combination in range(combinations):
        list_config.append([random.sample(CMElons,1),random.sample(CMElats,1),random.sample(CMEtilts,1),random.sample(heights,1),random.sample(ks,1),random.sample(angs,1),nleg, ncirc, ncross])
    dic_parameters['config'] = list_config
    return dic_parameters

## Función ajuste del centro del sol

In [46]:
def center_rSun_pixel(headers,plotranges, sat):
    x_cS = (headers[sat]['CRPIX1']*plotranges[sat][sat]*2)/headers[sat]['NAXIS1'] - plotranges[sat][sat]
    y_cS = (headers[sat]['CRPIX2']*plotranges[sat][sat]*2)/headers[sat]['NAXIS2'] - plotranges[sat][sat]
    return x_cS, y_cS

## Función Kernel Density Estimation

In [47]:
def xi_yi_zi_KDE(x, y, nbins=300): 
    
    #representa cada punto discreto por una campana de Gauss:
    k = kde.gaussian_kde([x,y]) 
    #nbins es el ancho de cada campana
    xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j] 
    zi = k(np.vstack([xi.flatten(), yi.flatten()])) 
    return  xi, yi, zi             
     

## Función para guardar configuración en .dat

In [48]:
# def save_config(*parameters):
    
#     date_str = datetime.datetime.strftime(datetime.datetime.now(), '%Y_%m_%d_')
#     total_models = 0
    
#     for parameter in parameters:
#         total_models *= len(parameter)
        
#     config_name = date_str + '{:04d}'.format(total_models+1)
    
#     #texto plano:
#     with open('/gehme/projects/2020_gcs_with_ml/data/%s.dat' %config_name, 'w') as config_file:
#         config_file.writelines("%s" % parameters[i] for i in range(len(parameters)))
        
  

In [49]:
def save_config(diccionario):
    
    date_str = datetime.datetime.strftime(datetime.datetime.now(), '%Y_%m_%d_')
    
    total_models = len(diccionario['config'])
        
    config_name = date_str + '{:04d}'.format(total_models)
    
    with open('/gehme/projects/2020_gcs_with_ml/data/forwardGCS_test/%s.json' %config_name, 'w') as json_file:
        json.dump(diccionario,json_file)

## Función forwardGCS


In [50]:
def forwardGCS(dict_parameters, headers, size_occ):
    
    # Get the location of sats and the range of each image:
    satpos, plotranges = processHeaders(headers)
    
    dict_parameters['satpos'] = satpos
    dict_parameters['plotranges'] = plotranges
    # creo Set de imágenes de CMEs para todas las combinaciones de los parámetros de entrada para cada vista de SATPOS:
    for config in dict_parameters['config']:
        clouds = getGCS(config[0], config[1], config[2], config[3], config[4], config[5], satpos, config[6], config[7], config[8])
        #with open('/gehme/projects/2020_gcs_with_ml/data/%s.dat' %config_name, 'a') as config_file:
        for sat in range(len(satpos)):
            #creo figura:
            f = plt.figure(figsize=(3.556,3.556), facecolor='black')
            ax = f.add_subplot()
            x = clouds[sat,:,1]
            y = clouds[0,:,2]
            #calculo el centro y el radio del sol en coordenada de plotrange
            x_cS, y_cS= center_rSun_pixel(headers,plotranges, sat)
            #mapeo la matriz [zi,xi], con límites [xi,yi]:
#             xi, yi, zi = xi_yi_zi_KDE(x, y)
#             plt.pcolormesh(xi, yi, zi.reshape(xi.shape), cmap='gist_yarg', facecolor='black') 
            #                                 plt.imshow(ims[sat], vmin=-10, vmax=10, cmap=cm.binary, zorder=0, extent=plotranges[sat])
            #imagen de puntos de la CME:
            plt.scatter(clouds[sat,:,1], clouds[0,:,2], s=5, c='purple', linewidths=0) 
            #circulos occulter y limbo:
            occulter = plt.Circle((x_cS, y_cS), size_occ[sat], fc='white')
            limbo = plt.Circle((x_cS, y_cS), 1, ec='black', fc='white')
            ax.add_artist(occulter)
            ax.add_artist(limbo)
            #tamaño de la imagen referenciado al tamaño de la imagen del sol:
            plt.xlim(plotranges[sat][0],plotranges[sat][1])
            plt.ylim(plotranges[sat][2],plotranges[sat][3])
            plt.axis('off')
            #guardo cada vista CME
            f.savefig('/gehme/projects/2020_gcs_with_ml/data/forwardGCS_test/{:08.3f}_{:08.3f}_{:08.3f}_{:08.3f}_{:08.3f}_{:08.3f}_sat{}.png'.format(config[0], config[1], config[2], config[3], config[4], config[5], sat+1),facecolor=f.get_facecolor())
#             plt.show()
            plt.close(f)

# DATOS DE ENTRADA

In [61]:
CMElon_val_ini = 60
CMElon_val_end = 65
CMElon_step_size = 1
CMElons = np.arange(CMElon_val_ini, CMElon_val_end, CMElon_step_size).tolist()

CMElat_val_ini = 20
CMElat_val_end = 25
CMElat_step_size = 1
CMElats = np.arange(CMElat_val_ini, CMElat_val_end, CMElat_step_size).tolist()

CMEtilt_val_ini = 70
CMEtilt_val_end = 75
CMEtilt_step_size = 1
CMEtilts = np.arange(CMEtilt_val_ini, CMEtilt_val_end, CMEtilt_step_size).tolist()

height_val_ini = 6
height_val_end = 11
height_step_size = 1
heights = np.arange(height_val_ini, height_val_end, height_step_size).tolist()

k_val_ini = 0.1
k_val_end = 0.6
k_step_size = 0.1
ks = np.arange(k_val_ini, k_val_end, k_step_size).tolist()

ang_val_ini = 50
ang_val_end = 55
ang_step_size = 1
angs = np.arange(ang_val_ini, ang_val_end, ang_step_size).tolist()

nleg = 5
ncirc = 20
ncross = 30


In [62]:
print(CMElons, CMElats, CMEtilts, heights, ks, angs)

[60, 61, 62, 63, 64] [20, 21, 22, 23, 24] [70, 71, 72, 73, 74] [6, 7, 8, 9, 10] [0.1, 0.2, 0.30000000000000004, 0.4, 0.5] [50, 51, 52, 53, 54]


## Headers

In [52]:
# flag if using LASCO data from ISSI which has STEREOlike headers already
ISSIflag = False

# Read in your data
DATA_PATH = '/gehme/data'
secchipath = DATA_PATH+'/stereo/secchi/L0'
lascopath = DATA_PATH+'/soho/lasco/level_1/c2'

CorA =  secchipath+'/a/img/cor2/20101214/level1/20101214_162400_04c2A.fts'
CorB = secchipath+'/b/img/cor2/20101214/level1/20101214_162400_04c2B.fts'
LascoC2 = lascopath+'/20101214/25354684.fts'
# mainpath  = '/gehme/data'
# eventpath = 'test_event/'
# c2path    = mainpath+eventpath+'c2/' 
# c3path    = mainpath+eventpath+'c3/' 
# cor2apath = mainpath+eventpath+'cor2/a/' 
# cor2bpath = mainpath+eventpath+'cor2/b/'

# ISSI data (hierarchy folders)
 
# fnameA2 = getFile(CorA)
# fnameB2 = getFile(CorB)
# fnameL2  = getFile(LascoC2)

# Non ISSI data (all in one folder)
# '''thisPath = mainpath+eventpath
# fnameA1 = getFile(thisPath, '20130522_080', ext='A.fts')
# fnameB1 = getFile(thisPath, '20130522_080', ext='B.fts')
# fnameL1  = getFile(thisPath, '20130522_075', ext='C2.fts') 
# fnameA2 = getFile(thisPath, '20130522_132', ext='A.fts')
# fnameB2 = getFile(thisPath, '20130522_132', ext='B.fts')
# fnameL2  = getFile(thisPath, '20130522_132', ext='C2.fts')'''

# STEREO A
# ima1, hdra1 = sunpy.io.fits.read(fnameA1)[0]
# smap_SA1 = sunpy.map.Map(ima1, hdra1)
ima2, hdra2 = sunpy.io.fits.read(CorA)[0]
smap_SA2 = sunpy.map.Map(ima2, hdra2)

# # STEREO B
# imb1, hdrb1 = sunpy.io.fits.read(fnameB1)[0]
# smap_SB1 = sunpy.map.Map(imb1, hdrb1)
imb2, hdrb2 = sunpy.io.fits.read(CorB)[0]
smap_SB2 = sunpy.map.Map(imb2, hdrb2)

# # LASCO

if ISSIflag:
#     imL1, hdrL1 = sunpy.io.fits.read(fnameL1)[0]
    imL2, hdrL2 = sunpy.io.fits.read(LascoC2)[0]
    
else:
#     with fits.open(fnameL1) as myfitsL1:
#         imL1 = myfitsL1[0].data
#         myfitsL1[0].header['OBSRVTRY'] = 'SOHO'
#         coordL1 = get_horizons_coord(-21, datetime.datetime.strptime(myfitsL1[0].header['DATE-OBS'], "%Y-%m-%dT%H:%M:%S.%f"), 'id')
#         coordL1carr = coordL1.transform_to(sunpy.coordinates.frames.HeliographicCarrington)
#         coordL1ston = coordL1.transform_to(sunpy.coordinates.frames.HeliographicStonyhurst)
#         myfitsL1[0].header['CRLT_OBS'] = coordL1carr.lat.deg
#         myfitsL1[0].header['CRLN_OBS'] = coordL1carr.lon.deg
#         myfitsL1[0].header['HGLT_OBS'] = coordL1ston.lat.deg
#         myfitsL1[0].header['HGLN_OBS'] = coordL1ston.lon.deg
#         hdrL1 = myfitsL1[0].header
    with fits.open(LascoC2) as myfitsL2:
        imL2 = myfitsL2[0].data
        myfitsL2[0].header['OBSRVTRY'] = 'SOHO'
        coordL2 = get_horizons_coord(-21, datetime.datetime.strptime(myfitsL2[0].header['DATE-OBS'], "%Y-%m-%dT%H:%M:%S.%f"), 'id')
        coordL2carr = coordL2.transform_to(sunpy.coordinates.frames.HeliographicCarrington)
        coordL2ston = coordL2.transform_to(sunpy.coordinates.frames.HeliographicStonyhurst)
        myfitsL2[0].header['CRLT_OBS'] = coordL2carr.lat.deg
        myfitsL2[0].header['CRLN_OBS'] = coordL2carr.lon.deg
        myfitsL2[0].header['HGLT_OBS'] = coordL2ston.lat.deg
        myfitsL2[0].header['HGLN_OBS'] = coordL2ston.lon.deg
        hdrL2 = myfitsL2[0].header
# smap_L1 = sunpy.map.Map(imL1, hdrL1)
smap_L2 = sunpy.map.Map(imL2, hdrL2)

headers = [hdra2, hdrL2, hdrb2]


INFO: Obtained JPL HORIZONS location for SOHO (spacecraft) (-21) [sunpy.coordinates.ephemeris]


## Tamaño de los occulters referenciados al RSUN

In [53]:
cor2 = 2
c3 = 3.7
size_occ = [cor2, c3, cor2]

## Prueba

In [68]:
dic_parameters = dic_Parameters(CMElons, CMElats, CMEtilts, heights, ks, angs, nleg, ncirc, ncross)

In [70]:
print(type(angs[3]))

<class 'int'>


In [69]:
forwardGCS(dic_parameters, headers, size_occ)

TypeError: can't multiply sequence by non-int of type 'float'